In [9]:
#import the required packages/libs
from craigslist import CraigslistForSale
import pandas as pd
import webbrowser
import smtplib
import time
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [10]:
#define the send email function, so that lists of results can be sent via email
def send_email(body, sender, recipients, subject):
    msg = 'Subject: {}\n\n{}'.format(subject, body)
    smtpserver = 'smtp.gmail.com:587'
    try:
        server = smtplib.SMTP(smtpserver)
        server.ehlo()
        server.starttls()
        server.login('thefifthfox0@gmail.com', 'computerfox')
        server.sendmail(sender, recipients, msg)
    except Exception as e:
        print(e)

#Search craigslist for cars and perform the necessary functions based on the parameters provided
def search_craigslist_for_cars(loc,car,minyear,maxyear,minprice,maxprice,openlinks,email,printheader):
    
    #Declare some initial variables
    msg = []
    already_checked = ['https://anchorage.craigslist.org/cto/d/1978-porsche-911-sc-only-5000/6680836329.html','https://austin.craigslist.org/cto/d/1976-porsche-911-convertible/6664202810.html','https://bakersfield.craigslist.org/cto/d/911-porsche-convertible/6686260433.html','https://bham.craigslist.org/cto/d/1983-porsche-911-bridgestone/6687907347.html','https://boise.craigslist.org/cto/d/rare-1970-porsche-911-body/6649790854.html','https://charleston.craigslist.org/cto/d/1986-porsche-911-conv/6649734485.html','https://cleveland.craigslist.org/cto/d/1983-porsche-911-goodyear/6687914825.html','https://columbia.craigslist.org/cto/d/1983-burgundy-porsche-911/6687196348.html','https://daytona.craigslist.org/cto/d/1986-porsche-911-carrera-conv/6649761594.html','https://eastoregon.craigslist.org/cto/d/early-porsche-911-project-wtb/6679741734.html','https://fargo.craigslist.org/cto/d/1983-porsche-911-garaged-with/6682299789.html','https://flint.craigslist.org/cto/d/1983-porsche-911-burgundy/6687285576.html','https://grandrapids.craigslist.org/cto/d/porsche-911-project-or-fixer/6685967106.html','https://greenville.craigslist.org/cto/d/1986-porsche-911-conv/6649701036.html','https://houston.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678163728.html','https://hudsonvalley.craigslist.org/cto/d/1982-porsche-911-slant-nose/6681725992.html','https://inlandempire.craigslist.org/cto/d/1983-porsche-911-driver-airbag/6687979723.html','https://lansing.craigslist.org/cto/d/911-porsche-looking-for-older/6687104481.html','https://lasvegas.craigslist.org/cto/d/1987-porsche-911-carrera/6687002558.html','https://lasvegas.craigslist.org/cto/d/1969-porsche-911-baja-race/6668904623.html','https://lasvegas.craigslist.org/cto/d/1977-porsche-911-sc-targa/6680151081.html','https://monterey.craigslist.org/cto/d/red-1977-porsche-911-sc-targa/6680149748.html','https://monterey.craigslist.org/ctd/d/1984-porsche-911-cab-guards/6677467366.html','https://nmi.craigslist.org/cto/d/69-porsche-911-irish-green/6656476893.html','https://norfolk.craigslist.org/cto/d/1972-porsche-911/6663694575.html','https://orlando.craigslist.org/cto/d/1991-porsche-911-carrera-2/6680295712.html','https://pensacola.craigslist.org/cto/d/1983-porsche-911-manual/6684020499.html','https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html','https://philadelphia.craigslist.org/cto/d/1974-porsche-911-targa/6670015872.html','https://reno.craigslist.org/cto/d/porsche-911/6683051897.html','https://sacramento.craigslist.org/cto/d/1967-porsche-911/6682711263.html']
    
    #Pull data based on parameters
    cars = CraigslistForSale(site=loc, area='', category='cars-trucks', 
                         filters={'min_price': minprice, 'max_price': maxprice, 'min_year':minyear, 'max_year':maxyear,'query':car})
    
    #Print title if desired
    if printheader == True:
        header = ''
        header = str(cars.filters['min_auto_year']) + ' till ' + str(cars.filters['max_auto_year']) + ' ' +cars.filters['query'] + ' between ' + str(cars.filters['min_price']) + ' and ' + str(cars.filters['max_price']) + ' in the ' + loc + ' area '
        print(header)
    
    #Loop through the results
    for x in cars.get_results():
        
        #Temporary holding of URL from the search results as a list to use as filter against the already_checked list
        temp = []
        temp.append(x['url'])
        
        #Only return results if url contains one of the words from the search critera AND
        #Also only return results that are not in the already_checked list
        if (len(set(car.split(' ')) & set( x['url'].split('/')[5].split('-'))) > 1) and (len(set(already_checked).intersection(temp)) == 0):
            msg.append('Posted: {} Price: {} Link: {}'.format(x['datetime'], x['price'], x['url']))
            
            #Open urls in webbrowser if desired
            if openlinks == True:
                webbrowser.open(x['url'])
    
    #Send email with search results if desired and search results exist
    if email == True and len(msg) > 0:
        send_email('\n'.join(msg), 'thefifthfox0@gmail.com', ['Qman2386@gmail.com'], header)
    
    #Print out the search results to the console 
    for _ in msg:
        print(_)

In [11]:
#solve for if link contains a word with no space like 
#send email based on individual location search or all sites
#create way to filter without copy pasting the links into already_checked
#detect new posts on https://forums.pelicanparts.com/porsche-cars-sale/
#setup a server to have it run every half hour
search_craigslist_for_cars('philadelphia','porsche 911',1900,1998,0,30000,False,False,True) 
#search_craigslist_for_cars('philadelphia','porsche 911',1900,1998,'','',False,False,True) 
#search_craigslist_for_cars('philadelphia','nissan skyline',1900,2018,'','',True,False,True)

1900 till 2018 nissan skyline between  and  in the philadelphia area 
Posted: 2018-09-05 15:33 Price: $17500 Link: https://philadelphia.craigslist.org/cto/d/nissan-skyline-r32-gtr-rwd/6683710756.html
Posted: 2018-08-30 15:02 Price: None Link: https://philadelphia.craigslist.org/ctd/d/1993-nissan-skyline-gts25t/6685024460.html
Posted: 2018-08-30 15:02 Price: None Link: https://philadelphia.craigslist.org/ctd/d/1992-nissan-skyline-gts-rb25/6685024665.html
Posted: 2018-08-24 16:17 Price: None Link: https://philadelphia.craigslist.org/ctd/d/1992-nissan-skyline-gtst-type/6679892707.html
Posted: 2018-08-24 16:16 Price: None Link: https://philadelphia.craigslist.org/ctd/d/1992-nissan-skyline-gtst-type/6679892283.html
Posted: 2018-08-15 17:09 Price: $29000 Link: https://philadelphia.craigslist.org/cto/d/jdm-rhd-1991-nissan-skyline/6672018702.html
Posted: 2018-08-08 15:05 Price: $32995 Link: https://philadelphia.craigslist.org/ctd/d/1992-nissan-skyline-gtr-jdm/6665675634.html


In [15]:
cars = ['porsche 911']
#,'impreza','mazda3','tsx','gti','accord','miata','corolla','fit','sonic']
sites = ['abilene','akroncanton','albany','albanyga','albuquerque','allentown','altoona','amarillo','ames','anchorage','annapolis','annarbor','appleton','asheville','ashtabula','athensga','athensohio','atlanta','auburn','augusta','austin','bakersfield','baltimore','batonrouge','battlecreek','beaumont','bellingham','bemidji','bend','bgky','bham','bigbend','billings','binghamton','bismarck','blacksburg','bloomington','bn','boise','boone','boston','boulder','bozeman','brainerd','brownsville','brunswick','buffalo','butte','capecod','carbondale','catskills','cedarrapids','cenla','centralmich','cfl','chambana','chambersburg','charleston','charlestonwv','charlotte','charlottesville','chattanooga','chautauqua','chicago','chico','chillicothe','cincinnati','clarksville','cleveland','clovis','cnj','collegestation','columbia','columbiamo','columbus','columbusga','cookeville','corpuschristi','corvallis','cosprings','csd','dallas','danville','dayton','daytona','decatur','delaware','delrio','denver','desmoines','detroit','dothan','dubuque','duluth','eastco','easternshore','eastidaho','eastky','eastnc','eastoregon','easttexas','eauclaire','elko','elmira','elpaso','enid','erie','eugene','evansville','fairbanks','fargo','farmington','fayar','fayetteville','fingerlakes','flagstaff','flint','florencesc','fortcollins','fortdodge','fortlauderdale','fortmyers','fortsmith','fortwayne','frederick','fredericksburg','fresno','gadsden','gainesville','galveston','glensfalls','goldcountry','grandforks','grandisland','grandrapids','greatfalls','greenbay','greensboro','greenville','gulfport','hanford','harrisburg','harrisonburg','hartford','hattiesburg','helena','hickory','hiltonhead','holland','honolulu','houma','houston','hudsonvalley','humboldt','huntington','huntsville','miami','imperial','indianapolis','inlandempire','iowacity','ithaca','jackson','jacksontn','jacksonville','janesville','jerseyshore','jonesboro','joplin','juneau','jxn','kalamazoo','kalispell','kansascity','kenai','keys','killeen','kirksville','klamath','knoxville','kokomo','kpr','ksu','lacrosse','lafayette','lakecharles','lakecity','lakeland','lancaster','lansing','laredo','lasalle','lascruces','lasvegas','lawrence','lawton','lewiston','lexington','limaohio','lincoln','littlerock','logan','longisland','losangeles','louisville','loz','lubbock','lynchburg','macon','madison','maine','mankato','mansfield','marshall','martinsburg','masoncity','mattoon','mcallen','meadville','medford','memphis','mendocino','merced','meridian','miami','milwaukee','minneapolis','missoula','mobile','modesto','mohave','monroe','monroemi','montana','monterey','montgomery','morgantown','moseslake','muncie','muskegon','myrtlebeach','nacogdoches','nashville','natchez','nd','nesd','newhaven','newjersey','newlondon','neworleans','newyork','nh','nmi','norfolk','northernwi','northmiss','northplatte','nwct','nwga','nwks','ocala','odessa','ogden','okaloosa','oklahomacity','olympic','omaha','oneonta','onslow','orangecounty','oregoncoast','orlando','ottumwa','outerbanks','owensboro','palmsprings','panamacity','parkersburg','pennstate','pensacola','peoria','philadelphia','phoenix','pittsburgh','plattsburgh','poconos','porthuron','portland','potsdam','prescott','providence','provo','pueblo','pullman','quadcities','quincy','racine','raleigh','rapidcity','reading','redding','reno','richmond','richmondin','rmn','roanoke','rochester','rockford','rockies','roseburg','roswell','sacramento','saginaw','salem','salina','saltlakecity','sanangelo','sanantonio','sandiego','sandusky','sanmarcos','santabarbara','santafe','santamaria','sarasota','savannah','scottsbluff','scranton','sd','seattle','seks','semo','sfbay','sheboygan','shoals','showlow','shreveport','sierravista','siouxcity','siouxfalls','siskiyou','skagit','slo','smd','southbend','southcoast','southjersey','spacecoast','spokane','springfield','springfieldil','statesboro','staugustine','stcloud','stgeorge','stillwater','stjoseph','stlouis','stockton','susanville','swks','swmi','swv','swva','syracuse','tallahassee','tampa','terrehaute','texarkana','texoma','thumb','tippecanoe','toledo','topeka','treasure','tricities','tucson','tulsa','tuscaloosa','tuscarawas','twinfalls','twintiers','up','utica','valdosta','ventura','vermont','victoriatx','visalia','waco','washingtondc','waterloo','watertown','wausau','wenatchee','westernmass','westky','westmd','westslope','wheeling','wichita','wichitafalls','williamsport','wilmington','winchester','winstonsalem','worcester','wv','wyoming','yakima','york','youngstown','yubasutter','yuma','zanesville']

In [16]:
#Search all sites for data
start = time.clock()
for y in cars:
    for x in sites:
        search_craigslist_for_cars(x,y,1900,1998,0,30000,False,False,True) 
end = time.clock()
print("It took approx " + color.BOLD + str(round((end-start)/60)) + " minutes " + color.END + "to run this program")

1900 till 1998 porsche 911 between 0 and 30000 in the abilene area 
1900 till 1998 porsche 911 between 0 and 30000 in the akroncanton area 
1900 till 1998 porsche 911 between 0 and 30000 in the albany area 
1900 till 1998 porsche 911 between 0 and 30000 in the albanyga area 
1900 till 1998 porsche 911 between 0 and 30000 in the albuquerque area 
1900 till 1998 porsche 911 between 0 and 30000 in the allentown area 
1900 till 1998 porsche 911 between 0 and 30000 in the altoona area 
1900 till 1998 porsche 911 between 0 and 30000 in the amarillo area 
1900 till 1998 porsche 911 between 0 and 30000 in the ames area 
1900 till 1998 porsche 911 between 0 and 30000 in the anchorage area 
1900 till 1998 porsche 911 between 0 and 30000 in the annapolis area 
1900 till 1998 porsche 911 between 0 and 30000 in the annarbor area 
1900 till 1998 porsche 911 between 0 and 30000 in the appleton area 
1900 till 1998 porsche 911 between 0 and 30000 in the asheville area 
1900 till 1998 porsche 911 betwe

1900 till 1998 porsche 911 between 0 and 30000 in the farmington area 
1900 till 1998 porsche 911 between 0 and 30000 in the fayar area 
1900 till 1998 porsche 911 between 0 and 30000 in the fayetteville area 
1900 till 1998 porsche 911 between 0 and 30000 in the fingerlakes area 
1900 till 1998 porsche 911 between 0 and 30000 in the flagstaff area 
1900 till 1998 porsche 911 between 0 and 30000 in the flint area 
1900 till 1998 porsche 911 between 0 and 30000 in the florencesc area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortcollins area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortdodge area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortlauderdale area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortmyers area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortsmith area 
1900 till 1998 porsche 911 between 0 and 30000 in the fortwayne area 
1900 till 1998 porsche 911 between 0 and 30000 in the frederick area 
1900 till 1998

1900 till 1998 porsche 911 between 0 and 30000 in the minneapolis area 
1900 till 1998 porsche 911 between 0 and 30000 in the missoula area 
1900 till 1998 porsche 911 between 0 and 30000 in the mobile area 
1900 till 1998 porsche 911 between 0 and 30000 in the modesto area 
1900 till 1998 porsche 911 between 0 and 30000 in the mohave area 
1900 till 1998 porsche 911 between 0 and 30000 in the monroe area 
1900 till 1998 porsche 911 between 0 and 30000 in the monroemi area 
1900 till 1998 porsche 911 between 0 and 30000 in the montana area 
1900 till 1998 porsche 911 between 0 and 30000 in the monterey area 
Posted: 2018-08-08 16:01 Price: $500 Link: https://monterey.craigslist.org/cto/d/unbolt-1970-porsche-911/6665908362.html
1900 till 1998 porsche 911 between 0 and 30000 in the montgomery area 
1900 till 1998 porsche 911 between 0 and 30000 in the morgantown area 
1900 till 1998 porsche 911 between 0 and 30000 in the moseslake area 
1900 till 1998 porsche 911 between 0 and 30000 in t

1900 till 1998 porsche 911 between 0 and 30000 in the seattle area 
1900 till 1998 porsche 911 between 0 and 30000 in the seks area 
1900 till 1998 porsche 911 between 0 and 30000 in the semo area 
1900 till 1998 porsche 911 between 0 and 30000 in the sfbay area 
1900 till 1998 porsche 911 between 0 and 30000 in the sheboygan area 
1900 till 1998 porsche 911 between 0 and 30000 in the shoals area 
1900 till 1998 porsche 911 between 0 and 30000 in the showlow area 
1900 till 1998 porsche 911 between 0 and 30000 in the shreveport area 
1900 till 1998 porsche 911 between 0 and 30000 in the sierravista area 
1900 till 1998 porsche 911 between 0 and 30000 in the siouxcity area 
1900 till 1998 porsche 911 between 0 and 30000 in the siouxfalls area 
1900 till 1998 porsche 911 between 0 and 30000 in the siskiyou area 
1900 till 1998 porsche 911 between 0 and 30000 in the skagit area 
1900 till 1998 porsche 911 between 0 and 30000 in the slo area 
1900 till 1998 porsche 911 between 0 and 30000 

In [5]:
cars = CraigslistForSale(site='philadelphia', area='', category='cars-trucks', 
                         filters={'min_price': 1000, 'max_price': 5000, 'min_year':2000, 'max_year':2010,'query':'porsche'})

In [6]:
for x in cars.get_results():
    print(x)

{'id': '6688427651', 'repost_of': '6656845279', 'name': '2003 Mercedes C32 AMG - 350hp', 'url': 'https://philadelphia.craigslist.org/cto/d/2003-mercedes-c32-amg-350hp/6688427651.html', 'datetime': '2018-09-03 20:26', 'price': '$4550', 'where': 'Downingtown', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6679909652', 'repost_of': None, 'name': '2005 Chevrolet TrailBlazer LS EXTENDED SUV 4X4 - 3RD ROW SEATING - V8', 'url': 'https://philadelphia.craigslist.org/ctd/d/2005-chevrolet-trailblazer-ls/6679909652.html', 'datetime': '2018-08-24 16:33', 'price': '$4650', 'where': '8100 BRISTOL PIKE (Rt. 13) LEVITTOWN, PA', 'has_image': True, 'has_map': True, 'geotag': None}
{'id': '6679853389', 'repost_of': None, 'name': '2009 Chevrolet Impala LT - Leather - Super Clean - Runs Strong!', 'url': 'https://philadelphia.craigslist.org/ctd/d/2009-chevrolet-impala-lt/6679853389.html', 'datetime': '2018-08-24 15:40', 'price': '$4350', 'where': '8100 BRISTOL PIKE (Rt. 13) LEVITTOWN, PA', 'has